<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing
A notebook for testing an exported model. Ideally, this can be considered a part of a model evaluation pipeline, in which a model can be evaluated in greater depth.

All notebooks in this project are to be considered development environments, rather than bona fide scripts that, when run, will produce the end product. Therefore, certain code blocks and documentation are added for developer convenience. 

## Setup

### Notebook Setup 

In [ ]:
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

In [ ]:
from fastai.vision.all import *
from google.colab import drive

import numpy as np 
import os 
import pandas as pd

drive.mount('/content/drive')

In [ ]:
%rm -rf /content/fyp/

In [3]:
# Import fyputil library
%cd /content
!git clone https://github.com/WRFitch/fyp.git
%cd fyp/src/fyputil
import constants as c
import fyp_utils as fyputil
%cd /content

/content
Cloning into 'fyp'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (313/313), done.
remote: Total 1351 (delta 258), reused 114 (delta 65), pack-reused 971
Receiving objects: 100% (1351/1351), 171.77 MiB | 25.65 MiB/s, done.
Resolving deltas: 100% (828/828), done.
/content/fyp/src/fyputil
/content


### Data Setup 

In [4]:
err_headers = [c.lon, c.lat] + c.ghg_bands

ghg_df = pd.read_csv(c.ghg_csv)
dnorm_ghg_df = pd.read_csv(c.ghg_csv)
ghg_df = fyputil.normGhgDf(ghg_df)
ghg_df

,Unnamed: 0,SO2_column_number_density,longitude,latitude,CH4_column_volume_mixing_ratio_dry_air,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density
0,0,47.938168,-0.795009,51.109648,10.531718,37.153821,9.913828,2.705834,6.081533
1,1,43.566527,-0.786026,51.109648,10.337841,39.323846,12.108644,4.124599,6.106126
2,2,46.108674,-0.777043,51.109648,8.525499,42.313591,12.558397,3.930123,4.485217
3,3,47.704085,-0.768060,51.109648,7.223523,41.804573,14.744170,3.580968,1.337698
4,4,51.427614,-0.759076,51.109648,8.900317,42.469231,19.928847,3.831177,0.821039
...,...,...,...,...,...,...,...,...,...
10792,10792,76.931755,0.363818,51.864233,78.159492,57.031807,47.757274,19.594411,98.302537
10793,10793,75.075876,0.372801,51.864233,75.768332,57.519190,40.399468,20.806978,94.711922
10794,10794,76.443141,0.381784,51.864233,79.216157,60.948625,40.913113,21.013862,95.276043
10795,10795,85.876993,0.390767,51.864233,77.271962,59.415460,35.553224,19.853508,94.502509


### Selecting Optimal Model

In [ ]:
# move through each model in model_dir and find the one with the best RMSE. 
# As of 10/03/21, this is mrghg_060321-resnet152_increased_dataset_size_to_4k.pkl
for root, _, files in os.walk(c.model_dir, topdown=True):
    for name in files:
      try:
        full_path = os.path.join(root, name)
        test_learner = load_learner(full_path)
      except Exception:
        print(Exception)
        print(f"model appears to have died. skipping... {full_path}")
        continue

      print(full_path)
      # Commented out because if it's unnecessarily run it'll take hours to complete. 
      # Only uncomment this if you have that time to spare. 
      # We're only testing 10% of the data, or otherwise we'll really be here all day. 
      #rmse = getModelRmse(test_learner, 10)
      print(rmse)


### Get model predictions

In [6]:
def getGhgsAsArr(img_path):
  return fyputil.getGhgsAsArr(img_path, ghg_df)

def getModelRmse(model, modulus=1, err_df=None):
  # Return a RMSE value for each GHG in the model's predicted values. 
  if err_df == None:
    err_df = getErrs(model, ghg_df, modulus)

  rtnval = []
  for col in err_headers[2:]:
    rtnval.append( math.sqrt( err_df[col].apply(lambda x:x**2) .mean()))
  return rtnval

def getPreds(model, ghg_df, modulus=1):
  # Return a dataframe containing the model predictions from the given dataframe
  pred_df = pd.DataFrame(columns=err_headers)
  mod = 0
  for idx, row in ghg_df.iterrows():
    coords = (row.longitude, row.latitude)
    if not fyputil.imgExported(coords): continue
    if mod % modulus == 0 :
      prediction = model.predict(fyputil.getFilepath(coords))[0]
      pred_df.loc[len(pred_df)] = list(coords) + list(prediction)
    mod += 1
  return pred_df

def getErrs(model, df, modulus=1, preds_df=None):
  # Return a dataframe containing the difference between each prediction and the original value
  if preds_df.empty:
    preds_df = getPreds(model, df, modulus)
  return getDiffs(preds_df, df)

def getDiffs(pred_df, actual_df):
  # Return a dataframe containing the differences between coordinate-indexed values in two dataframes. 
  diffs = pd.DataFrame(columns = err_headers)
  for idx, row in pred_df.iterrows():
    coords = (row.longitude, row.latitude)
    # Keeping this in index lockstep would be an order of magnitude more efficient than this bodged lookup. 
    actual = fyputil.getValAt(coords, actual_df)[c.ghg_bands].squeeze()
    prediction = row[2:]
    if actual.empty: continue
    differences = [pred - act for pred, act in zip(prediction, actual)]
    diffs.loc[len(diffs)] = list(coords) + differences
  return diffs

In [7]:
#model_name = c.model_name
model_name = "140321_add-normalisation_bs-128_trained-some-more"
best_model = load_learner(f"{c.model_dir}/{model_name}.pkl")

In [ ]:
valid_df = getModelRmse(best_model, 1000)
valid_df

In [ ]:
preds = getPreds(best_model, ghg_df)

In [ ]:
preds

In [ ]:
errs = getErrs(best_model, ghg_df, preds_df=preds)

In [ ]:
errs

### Save model predictions

In [ ]:
# Commented out so they aren't accidentally overwritten 
preds.to_csv(f"{c.data_dir}/best_preds-{model_name}.csv")
errs.to_csv(f"{c.data_dir}/pred_errs-{model_name}.csv")

### Retrieve model predictions

In [ ]:
preds = pd.read_csv(f"{c.data_dir}/best_preds-{model_name}.csv")
errs = pd.read_csv(f"{c.data_dir}/pred_errs-{model_name}.csv")

In [ ]:
preds

In [ ]:
errs

## Testing

### Basic stat testing 
- Data exploration 
- RMSE per GHG
- Extract outliers & view images 

In [ ]:
model_stats = pd.DataFrame(columns = ["stat"] + c.ghg_bands)

In [ ]:
def getRmse(series): 
  return np.sqrt(np.mean(series**2))

In [ ]:
# Define aggregate metrics 
# TODO remove multiple iterations through errors, improve bigO 
means = [errs[ghg].mean() for ghg in c.ghg_bands ]
stdevs = [errs[ghg].std() for ghg in c.ghg_bands ]
rmse = [getRmse(errs[ghg]) for ghg in c.ghg_bands ]
mae = [errs[ghg].abs().mean() for ghg in c.ghg_bands ]

model_stats.loc[1] = ["Mean"] + means
model_stats.loc[2] = ["Standard Deviation"] + stdevs 
model_stats.loc[3] = ["RMSE"] + rmse
model_stats.loc[4] = ["MAE"] + mae

model_stats["avg"] = model_stats.mean(axis=1)

In [ ]:
model_stats

,stat,CO_column_number_density,tropospheric_HCHO_column_number_density,tropospheric_NO2_column_number_density,O3_column_number_density,SO2_column_number_density,CH4_column_volume_mixing_ratio_dry_air,avg
1,Mean,1.794107,-2.249522,-12.430723,-15.430360,6.025805,-11.210091,-5.583464
2,Standard Deviation,16.094458,14.553593,20.634910,17.543104,13.214439,16.556573,16.432846
3,RMSE,16.193394,14.725741,24.089048,23.362951,14.522921,19.994009,18.814677
4,MAE,12.932604,11.779999,16.725221,18.843600,11.788544,15.293509,14.560579


#### Plot raw stats 

In [ ]:
# Merge ghg and recalculate predictions 
errcols = [f"{ghg}_err" for ghg in c.ghg_bands]
combi_df = ghg_df.merge(errs, how="inner", on=[c.lon, c.lat], suffixes=("_orig", "_err"))
for ghg in c.ghg_bands:
  combi_df[f"{ghg}_pred"] = combi_df[f"{ghg}_orig"] + combi_df[f"{ghg}_err"]

combi_df["errsum"] = combi_df[errcols].sum(axis=1)
combi_df["errabs"] = combi_df[errcols].abs().sum(axis=1)

In [ ]:
combi_df

In [ ]:
for ghg in c.ghg_bands:
  combi_df.plot(x = f"{ghg}_orig", y = f"{ghg}_pred", kind = "scatter")
  plt.show()

### Find and process Outliers 
- Percentile 
  - 1.5*IQR for weak outliers
  - 3*IQR for strong outliers
- Linear regression 
- Standard deviation +- 2 (or 3) 
- Normal probability plot 



In [ ]:
# Individual outlier bands 
outliers = []
for ghg in c.ghg_bands:
  ghg_outliers = []
  q1 = combi_df[f"{ghg}_err"].quantile(0.25)
  q3 = combi_df[f"{ghg}_err"].quantile(0.75)
  iqr = q3 - q1
  lbound = q1 - 1.5*iqr
  ubound = q3 + 1.5*iqr
  ghg_outliers = combi_df.loc[(combi_df[f"{ghg}_err"] < lbound) | (combi_df[f"{ghg}_err"] > ubound)]
  outliers.append(ghg_outliers)


In [ ]:
all_outliers = pd.concat(outliers, join="inner").drop_duplicates()
all_outliers["errsum"] = all_outliers[errcols].abs().sum(axis=1)
all_outliers

In [ ]:
errcols = [f"{ghg}_err" for ghg in c.ghg_bands]
multiple_outliers = pd.concat(outliers, join="inner")
multiple_outliers = multiple_outliers[multiple_outliers.duplicated()]
multiple_outliers = multiple_outliers.drop_duplicates()
multiple_outliers["errsum"] = multiple_outliers[errcols].abs().sum(axis=1)
multiple_outliers

In [ ]:
multiple_outliers.nlargest(10, ['errsum'])

In [ ]:
# Show largest overpredictors 
for idx, row in combi_df.nlargest(50, ['errsum']).iterrows():
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

In [ ]:
# show underpredictors
for idx, row in combi_df.nsmallest(50, ['errsum']).iterrows():
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

In [ ]:
# show best predictions 
for idx, row in combi_df.nsmallest(50, ['errabs']).iterrows():
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

### Sample images vs predictions 
what regions are easier to predict than others? 

create accuracy heatmap 

In [ ]:
# Show outliers 
for idx, row in combi_df.nlargest(20, [f"{c.SO2_band}_err"]).iterrows():
  if row[f"{c.SO2_band}_pred"] in [0, 100]: continue
  coords = (row[c.lon], row[c.lat])
  print(coords)
  print(row)
  img_path = fyputil.getFilepath(coords)
  display(Image.open(img_path))

###Testing against  other areas
Export and test areas from a few different places
- desert
- tundra
- creepy american robo-farms
- other major cities
  - manchester
  - paris
  - tokyo
  - new york 

### Plot errors on folium heatmap 

###Experiment with facet implementation
https://github.com/BCG-Gamma/facet

### bicubic/linear/non-grid-based interpolation